# 6 WorkFlow  Gerencial, futuro=JULIO - Multiple Seeds Version

### 6.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer
Modificado para ejecutar con 5 semillas diferentes

#### 6.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


cp: cannot stat '/content/buckets/b1/kaggle/kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


## 6.3  Workflow con 5 Semillas

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Nov 21 03:28:06 PM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660358,35.3,1454505,77.7,1454505,77.7
Vcells,1226521,9.4,8388608,64.0,1975136,15.1


In [ ]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




#### Parametros Globales
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM_GLOBAL <- list()
PARAM_GLOBAL$experimento_base <- 6340
PARAM_GLOBAL$dataset <- "gerencial_competencia_2025.csv.gz"

# Vector de 5 semillas diferentes
PARAM_GLOBAL$semillas <- c(105727, 123457, 150011, 193939, 409993)

# Lista para almacenar resultados de todas las semillas
resultados_totales <- list()

## 6.3 Loop Principal - Iteración Automática sobre las 5 Semillas

Este loop ejecuta todo el workflow completo para cada una de las 5 semillas de manera automática.

In [ ]:
# Loop automático sobre todas las semillas
for (seed_idx in 1:length(PARAM_GLOBAL$semillas)) {

  cat("\n\n========================================\n")
  cat("PROCESANDO SEMILLA ", seed_idx, " de ", length(PARAM_GLOBAL$semillas), "\n")
  cat("Semilla: ", PARAM_GLOBAL$semillas[seed_idx], "\n")
  cat("========================================\n\n")

  # Inicializar PARAM para esta semilla
  PARAM <- list()
  PARAM$semilla_primigenia <- PARAM_GLOBAL$semillas[seed_idx]
  PARAM$experimento <- PARAM_GLOBAL$experimento_base + seed_idx - 1
  PARAM$dataset <- PARAM_GLOBAL$dataset
  PARAM$out <- list()
  PARAM$out$lgbm <- list()

  # ===================================================================
  # Carpeta del Experimento
  # ===================================================================
  setwd("/content/buckets/b1/exp")
  experimento_folder <- paste0("WF", PARAM$experimento, "_seed", seed_idx)
  dir.create(experimento_folder, showWarnings=FALSE)
  setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

  cat("Carpeta de trabajo: ", experimento_folder, "\n")

  # ===================================================================
  # Preprocesamiento del dataset
  # ===================================================================

  # Lectura del dataset
  dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

  # Catastrophe Analysis
  dataset[ foto_mes==202006, internet:=NA]
  dataset[ foto_mes==202006, mrentabilidad:=NA]
  dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
  dataset[ foto_mes==202006, mcomisiones:=NA]
  dataset[ foto_mes==202006, mactivos_margen:=NA]
  dataset[ foto_mes==202006, mpasivos_margen:=NA]
  dataset[ foto_mes==202006, mcuentas_saldo:=NA]
  dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
  dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
  dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
  dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
  dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
  dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

  # Data Drifting - sin codigo en esta primera version

  # Feature Engineering intra-mes
  atributos_presentes <- function( patributos )
  {
    atributos <- unique( patributos )
    comun <- intersect( atributos, colnames(dataset) )
    return(  length( atributos ) == length( comun ) )
  }

  if( atributos_presentes( c("foto_mes") ))
    dataset[, kmes := foto_mes %% 100]

  if( atributos_presentes( c("mpayroll", "cliente_edad") ))
    dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

  # Feature Engineering Historico
  # Asegúrate de cargar data.table
library(data.table)

# ---------------------------------------------------------------------------------------
# PASOS CRÍTICOS DE PREPARACIÓN
# ---------------------------------------------------------------------------------------

# CRÍTICO: Ordenar el dataset por cliente y por tiempo (foto_mes).
setkey(dataset, numero_de_cliente, foto_mes)

# ---------------------------------------------------------------------------------------
# 1. Definición Exhaustiva de Variables Lagueables
# ---------------------------------------------------------------------------------------

# Definición de columnas a excluir (IDs, target y categóricas no numéricas)
columnas_a_excluir <- c(
  "numero_de_cliente", "foto_mes", "clase_ternaria",
  "Master_status", "Visa_status" # Agrega aquí otras columnas categóricas si existen
)

# Filtramos TODAS las columnas que son numéricas y que no están en la lista de exclusión.
todas_las_variables_historicas <- names(dataset)[
    !names(dataset) %in% columnas_a_excluir &
    sapply(dataset, is.numeric)
]

# Definimos los Lags a aplicar
lags_a_aplicar <- c(1, 2, 3)

# ---------------------------------------------------------------------------------------
# 2. Aplicación Exhaustiva de Lags y Deltas
# ---------------------------------------------------------------------------------------

cat("💣 Ejecutando Experimento B1: FE Exhaustivo (Solo Lags y Deltas) sobre", length(todas_las_variables_historicas), "variables.\n")

columnas_previas <- ncol(dataset)

for (col in todas_las_variables_historicas) {

    # Control de errores (aunque ya se filtraron, es buena práctica)
    if (!is.numeric(dataset[[col]])) {
        cat(paste0("Advertencia: La columna '", col, "' no es numérica. Saltando.\n"))
        next
    }

    for (lag in lags_a_aplicar) {
        new_lag_col <- paste0(col, "_lag", lag)
        new_delta_col <- paste0(col, "_delta", lag)

        # Lag (Valor anterior)
        dataset[, (new_lag_col) := shift(get(col), n = lag, type = "lag"), by = numero_de_cliente]

        # Delta (Diferencia: valor actual - lag)
        dataset[, (new_delta_col) := get(col) - get(new_lag_col)]
    }
}

columnas_nuevas <- ncol(dataset) - columnas_previas
cat("\n[Resultado] Experimento B1 finalizado. Se crearon", columnas_nuevas, "nuevas variables históricas (Lags y Deltas).\n")
cat("El total de columnas ahora es:", ncol(dataset), ".\n")

  # ===================================================================
  # Modelado - Training Strategy
  # ===================================================================

  PARAM$trainingstrategy <- list()
  PARAM$trainingstrategy$validate <- c(202105)

  PARAM$trainingstrategy$training <- c(
    202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )

  PARAM$trainingstrategy$training_pct <- 1.0
  PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

  dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

  campos_buenos <- copy( setdiff(
      colnames(dataset), c("clase_ternaria","clase01","azar"))
  )

  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
  dataset[, azar:=runif(nrow(dataset))]

  dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
      (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
       azar < PARAM$trainingstrategy$training_pct ) ]

  if( !require("lightgbm")) install.packages("lightgbm")
  require("lightgbm")

  dtrain <- lgb.Dataset(
    data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
    label= dataset[fold_train == TRUE, clase01],
    free_raw_data= TRUE
  )

  dvalidate <- lgb.Dataset(
    data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
    label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
    free_raw_data= TRUE
  )

  # ===================================================================
  # Hyperparameter Tuning
  # ===================================================================

  if(!require("DiceKriging")) install.packages("DiceKriging")
  require("DiceKriging")

  if(!require("mlrMBO")) install.packages("mlrMBO")
  require("mlrMBO")

  PARAM$hipeparametertuning <- list()
  PARAM$hipeparametertuning$num_interations <- 10
  PARAM$lgbm <- list()

  PARAM$lgbm$param_fijos <- list(
    objective= "binary",
    metric= "auc",
    first_metric_only= TRUE,
    boost_from_average= TRUE,
    feature_pre_filter= FALSE,
    verbosity= -100,
    force_row_wise= TRUE,
    seed= PARAM$semilla_primigenia,
    max_bin= 31,
    learning_rate= 0.03,
    feature_fraction= 0.5,
    num_iterations= 2048,
    early_stopping_rounds= 200
  )

  PARAM$hipeparametertuning$hs <- makeParamSet(
    makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
    makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
  )

  EstimarGanancia_AUC_lightgbm <- function(x) {

    param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

    modelo_train <- lgb.train(
      data= dtrain,
      valids= list(valid = dvalidate),
      eval= "auc",
      param= param_completo,
      verbose= -100
    )

    AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]
    attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

    rm(modelo_train)
    gc(full= TRUE, verbose= FALSE)

    message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

    return(AUC)
  }

  configureMlr(show.learner.output = FALSE)

  obj.fun <- makeSingleObjectiveFunction(
      fn= EstimarGanancia_AUC_lightgbm,
      minimize= FALSE,
      noisy= FALSE,
      par.set= PARAM$hipeparametertuning$hs,
      has.simple.signature= FALSE
  )

  ctrl <- makeMBOControl(
      save.on.disk.at.time= 600,
      save.file.path= "HT.RDATA"
  )

  ctrl <- setMBOControlTermination(
      ctrl,
      iters= PARAM$hipeparametertuning$num_interations
  )

  ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

  surr.km <- makeLearner(
      "regr.km",
      predict.type= "se",
      covtype= "matern3_2",
      control= list(trace = TRUE)
  )

  if (!file.exists("HT.RDATA")) {
    bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
  } else {
    bayesiana_salida <- mboContinue("HT.RDATA")
  }

  tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
  setorder(tb_bayesiana, -y, -num_iterations)

  fwrite( tb_bayesiana,
    file="BO_log.txt",
    sep="\t"
  )

  PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
    1,
    setdiff(colnames(tb_bayesiana),
      c("y","dob","eol","error.message","exec.time","ei","error.model",
        "train.time","prop.type","propose.time","se","mean","iter")),
    with= FALSE
  ]

  print(PARAM$out$lgbm$mejores_hiperparametros)

  # ===================================================================
  # Produccion
  # ===================================================================

  PARAM$trainingstrategy$final_train <- c(
    202105, 202104, 202103, 202102, 202101,
    202012, 202011, 202010, 202009, 202008, 202007,
    202006, 202005
  )

  dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

  dfinal_train <- lgb.Dataset(
    data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
    label= dataset[fold_final_train == TRUE, clase01],
    free_raw_data= TRUE
  )

  fijos <- copy(PARAM$lgbm$param_fijos)
  fijos$num_iterations <- NULL
  fijos$early_stopping_rounds <- NULL

  param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

  final_model <- lgb.train(
    data= dfinal_train,
    param= param_final,
    verbose= -100
  )

  lgb.save(final_model, "modelo.txt")

  tb_importancia <- as.data.table(lgb.importance(final_model))
  fwrite( tb_importancia,
    file= "impo.txt",
    sep= "\t"
  )

  # ===================================================================
  # Scoring
  # ===================================================================

  PARAM$trainingstrategy$future <- c(202107)
  dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

  prediccion <- predict(
    final_model,
    data.matrix(dfuture[, campos_buenos, with= FALSE])
  )

  tb_prediccion <- dfuture[, list(numero_de_cliente)]
  tb_prediccion[, prob := prediccion]

  fwrite(tb_prediccion,
    file= "prediccion.txt",
    sep= "\t"
  )

  # ===================================================================
  # Curva de Ganancia
  # ===================================================================

  tb_prediccion[, clase_ternaria := dfuture$clase_ternaria ]
  tb_prediccion[, ganancia := -3000.0 ]
  tb_prediccion[clase_ternaria=="BAJA+2", ganancia := 117000.0 ]

  setorder( tb_prediccion, -prob )
  tb_prediccion[, gan_acum := cumsum(ganancia)]

  tb_prediccion[,
    gan_suavizada := frollmean(
      x= gan_acum,
      n= 400,
      align= "center",
      na.rm= TRUE,
      hasNA= TRUE
    )
  ]

  resultado <- list()
  resultado$ganancia_suavizada_max <- max( tb_prediccion$gan_suavizada, na.rm=TRUE )
  options(digits= 8)
  resultado$envios <- which.max( tb_prediccion$gan_suavizada)
  resultado$semilla <- PARAM$semilla_primigenia
  resultado$seed_idx <- seed_idx

  print(resultado)

  fwrite( tb_prediccion,
    file= "ganancias.txt",
    sep= "\t"
  )

  tb_prediccion[, envios:= .I]

  pdf("curva_de_ganancia.pdf")

  plot(
    x= tb_prediccion$envios,
    y= tb_prediccion$gan_acum,
    type= "l",
    col= "gray",
    xlim= c(0, 6000),
    ylim= c(0, 8000000),
    main= paste0("Seed ", seed_idx, " - Gan= ", as.integer(resultado$ganancia_suavizada_max), " envios= ", resultado$envios),
    xlab= "Envios",
    ylab= "Ganancia",
    panel.first= grid()
  )

  dev.off()

  if( !require("yaml")) install.packages("yaml")
  require("yaml")

  PARAM$resultado <- resultado

  write_yaml( PARAM, file="PARAM.yml")

  # ===================================================================
  # Guardar resultado y limpiar para siguiente iteración
  # ===================================================================

  if(!exists("resultados_totales")) resultados_totales <- list()
  resultados_totales[[seed_idx]] <- resultado

  rm(dataset, dtrain, dvalidate, dfinal_train, final_model, tb_prediccion)
  gc(full=TRUE, verbose=FALSE)

  cat("\n========================================\n")
  cat("Semilla ", seed_idx, " completada exitosamente\n")
  cat("Ganancia: ", resultado$ganancia_suavizada_max, "\n")
  cat("Envios: ", resultado$envios, "\n")
  cat("========================================\n\n")

} # Fin del loop sobre las semillas

cat("\n\n***************************************\n")
cat("TODAS LAS SEMILLAS PROCESADAS\n")
cat("***************************************\n")



PROCESANDO SEMILLA  1  de  5 
Semilla:  105727 

Carpeta de trabajo:  WF6340_seed1 
💣 Ejecutando Experimento B1: FE Exhaustivo (Solo Lags y Deltas) sobre 29 variables.

[Resultado] Experimento B1 finalizado. Se crearon 174 nuevas variables históricas (Lags y Deltas).
El total de columnas ahora es: 208 .


Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘para

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        203              679           1335
$ganancia_suavizada_max
[1] 5539200

$envios
[1] 1605

$semilla
[1] 105727

$seed_idx
[1] 1



Loading required package: yaml




Semilla  1  completada exitosamente
Ganancia:  5539200 
Envios:  1605 



PROCESANDO SEMILLA  2  de  5 
Semilla:  123457 

Carpeta de trabajo:  WF6341_seed2 
💣 Ejecutando Experimento B1: FE Exhaustivo (Solo Lags y Deltas) sobre 29 variables.

[Resultado] Experimento B1 finalizado. Se crearon 174 nuevas variables históricas (Lags y Deltas).
El total de columnas ahora es: 208 .


...

Computing y column(s) for design. Not provided.

Fri Nov 21 08:06:27 PM 2025 AUC 0.926910303462225

Fri Nov 21 08:10:58 PM 2025 AUC 0.927628678357958

Fri Nov 21 08:17:06 PM 2025 AUC 0.923021147265847

Fri Nov 21 08:26:07 PM 2025 AUC 0.934081302187873

Fri Nov 21 08:34:17 PM 2025 AUC 0.930330720877968

Fri Nov 21 08:39:54 PM 2025 AUC 0.932427462925346

Fri Nov 21 08:47:40 PM 2025 AUC 0.935185539358288

Fri Nov 21 08:51:48 PM 2025 AUC 0.920199990739305

[mbo] 0: num_leaves=120; min_data_in_leaf=3265 : y = 0.927 : 328.6 secs : initdesign

[mbo] 0: num_leaves=166; min_data_in_leaf=4635 : y = 0.928 : 270.7 secs : initdesign

[mbo] 0: num_leaves=13; min_data_in_leaf=6231 : y = 0.923 : 367.4 secs : initdesign

[mbo] 0: num_leaves=140; min_data_in_leaf=2277 : y = 0.934 : 541.1 secs : initdesign

[mbo] 0: num_leaves=216; min_data_in_leaf=5601 : y = 0.93 : 490.6 secs : initdesign

[mbo] 0: num_leaves=47; min_data_in_leaf=948 : y = 0.932 : 336.3 secs : initdesign

[mbo] 0: num_leaves=246; min_da

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        160              547           1676
$ganancia_suavizada_max
[1] 5440200

$envios
[1] 1713

$semilla
[1] 150011

$seed_idx
[1] 3


Semilla  3  completada exitosamente
Ganancia:  5440200 
Envios:  1713 



PROCESANDO SEMILLA  4  de  5 
Semilla:  193939 

Carpeta de trabajo:  WF6343_seed4 
💣 Ejecutando Experimento B1: FE Exhaustivo (Solo Lags y Deltas) sobre 29 variables.

[Resultado] Experimento B1 finalizado. Se crearon 174 nuevas variables históricas (Lags y Deltas).
El total de columnas ahora es: 208 .


Computing y column(s) for design. Not provided.

Fri Nov 21 10:25:04 PM 2025 AUC 0.932140582712431

Fri Nov 21 10:30:54 PM 2025 AUC 0.921733709398062

Fri Nov 21 10:38:59 PM 2025 AUC 0.932144944633797

Fri Nov 21 10:43:10 PM 2025 AUC 0.922925520528209

Fri Nov 21 10:51:34 PM 2025 AUC 0.930679003522419

Fri Nov 21 10:59:54 PM 2025 AUC 0.937842285002573

Fri Nov 21 11:02:16 PM 2025 AUC 0.923435865328027

Fri Nov 21 11:09:12 PM 2025 AUC 0.925738288744431

[mbo] 0: num_leaves=99; min_data_in_leaf=3016 : y = 0.932 : 518.9 secs : initdesign

[mbo] 0: num_leaves=211; min_data_in_leaf=7914 : y = 0.922 : 349.4 secs : initdesign

[mbo] 0: num_leaves=75; min_data_in_leaf=5120 : y = 0.932 : 485.4 secs : initdesign

[mbo] 0: num_leaves=16; min_data_in_leaf=5377 : y = 0.923 : 250.9 secs : initdesign

[mbo] 0: num_leaves=186; min_data_in_leaf=3596 : y = 0.931 : 503.8 secs : initdesign

[mbo] 0: num_leaves=239; min_data_in_leaf=1501 : y = 0.938 : 500.4 secs : initdesign

[mbo] 0: num_leaves=37; min_da

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        239             1501           1629
$ganancia_suavizada_max
[1] 5656200

$envios
[1] 1593

$semilla
[1] 193939

$seed_idx
[1] 4


Semilla  4  completada exitosamente
Ganancia:  5656200 
Envios:  1593 



PROCESANDO SEMILLA  5  de  5 
Semilla:  409993 

Carpeta de trabajo:  WF6344_seed5 
💣 Ejecutando Experimento B1: FE Exhaustivo (Solo Lags y Deltas) sobre 29 variables.

[Resultado] Experimento B1 finalizado. Se crearon 174 nuevas variables históricas (Lags y Deltas).
El total de columnas ahora es: 208 .


Computing y column(s) for design. Not provided.

Sat Nov 22 12:37:27 AM 2025 AUC 0.930054577702261

Sat Nov 22 12:45:21 AM 2025 AUC 0.933459225094603

Sat Nov 22 12:53:59 AM 2025 AUC 0.932549932256006

Sat Nov 22 01:01:45 AM 2025 AUC 0.92475215898331

Sat Nov 22 01:05:19 AM 2025 AUC 0.924689414422123

Sat Nov 22 01:09:17 AM 2025 AUC 0.917545258289496

Sat Nov 22 01:13:40 AM 2025 AUC 0.924750816853659

Sat Nov 22 01:20:53 AM 2025 AUC 0.937040027003649

[mbo] 0: num_leaves=70; min_data_in_leaf=5464 : y = 0.93 : 436.7 secs : initdesign

[mbo] 0: num_leaves=178; min_data_in_leaf=1877 : y = 0.933 : 474.3 secs : initdesign

[mbo] 0: num_leaves=247; min_data_in_leaf=2892 : y = 0.933 : 518.5 secs : initdesign

[mbo] 0: num_leaves=127; min_data_in_leaf=7680 : y = 0.925 : 465.9 secs : initdesign

[mbo] 0: num_leaves=160; min_data_in_leaf=4389 : y = 0.925 : 213.7 secs : initdesign

[mbo] 0: num_leaves=7; min_data_in_leaf=3677 : y = 0.918 : 238.0 secs : initdesign

[mbo] 0: num_leaves=34; min_data

## Resumen de Resultados de Todas las Semillas

In [ ]:
# Crear tabla resumen
setwd("/content/buckets/b1/exp")

tb_resumen <- data.table(
  seed_idx = sapply(resultados_totales, function(x) x$seed_idx),
  semilla = sapply(resultados_totales, function(x) x$semilla),
  ganancia = sapply(resultados_totales, function(x) x$ganancia_suavizada_max),
  envios = sapply(resultados_totales, function(x) x$envios)
)

# Agregar estadísticas
tb_resumen[, rank := rank(-ganancia)]

cat("\n\n========================================\n")
cat("RESUMEN FINAL DE LAS 5 SEMILLAS\n")
cat("========================================\n\n")
print(tb_resumen)

cat("\nESTADÍSTICAS:\n")
cat("Ganancia promedio: ", mean(tb_resumen$ganancia), "\n")
cat("Ganancia máxima: ", max(tb_resumen$ganancia), "\n")
cat("Ganancia mínima: ", min(tb_resumen$ganancia), "\n")
cat("Desviación estándar: ", sd(tb_resumen$ganancia), "\n")
cat("Coeficiente de variación: ", sd(tb_resumen$ganancia)/mean(tb_resumen$ganancia)*100, "%\n")
cat("Mejor semilla: ", tb_resumen[rank==1, semilla], " (seed_idx ", tb_resumen[rank==1, seed_idx], ")\n")

# Guardar resumen
fwrite(tb_resumen,
  file=paste0("resumen_5_seeds_exp", PARAM_GLOBAL$experimento_base, ".txt"),
  sep="\t"
)

# Guardar objeto completo
saveRDS(resultados_totales,
  file=paste0("resultados_completos_exp", PARAM_GLOBAL$experimento_base, ".rds")
)

cat("\nArchivos guardados:\n")
cat("- resumen_5_seeds_exp", PARAM_GLOBAL$experimento_base, ".txt\n")
cat("- resultados_completos_exp", PARAM_GLOBAL$experimento_base, ".rds\n")
cat("\nCada semilla tiene su carpeta individual con resultados detallados.\n")



RESUMEN FINAL DE LAS 5 SEMILLAS

   seed_idx semilla ganancia envios  rank
      <int>   <num>    <num>  <int> <num>
1:        1  153929  5733900   1439     5
2:        2  838969  6232800   1801     1
3:        3  922081  6085800   1160     2
4:        4  795581  5831700   1529     4
5:        5  194609  5832600   1314     3

ESTADÍSTICAS:
Ganancia promedio:  5943360 
Ganancia máxima:  6232800 
Ganancia mínima:  5733900 
Desviación estándar:  207770.24 
Coeficiente de variación:  3.4958381 %
Mejor semilla:  838969  (seed_idx  2 )

Archivos guardados:
- resumen_5_seeds_exp 6310 .txt
- resultados_completos_exp 6310 .rds

Cada semilla tiene su carpeta individual con resultados detallados.


In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Thu Nov 20 08:18:17 AM 2025"